
# <font color=blue><div align="center">ST7 SNCF : Jalon 1</div></font>

## <font color=blue><div align="center">"<em>Indian Railways</em>"</div></font>

## Modules

In [3]:
# Modules de base
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import datetime
%matplotlib inline
from dateutil import parser

# Module relatif à Gurobi
from gurobipy import *


# Module csv
import csv
from datetime import datetime

In [4]:
mini_instance="mini_instance.xlsx"
instance_simple="instance_WPY_simple.xlsx"
instance_realiste="instance_WPY_realiste_jalon1.xlsx"
test="mini_instance_test.xlsx"

## Traitement de données

In [5]:
def excel_to_dict(file :str)-> dict[str]:
    '''
    Takes excel file name in the repo
    and returns a dictionary[sheet_name]=Dataframe
    '''
    # Load the Excel file
    xl = pd.ExcelFile(file)
    
    # Get the names of the sheets
    sheet_names = xl.sheet_names
    
    # Initialize a dictionary to store the data
    data_dict = {}
    
    # Iterate over each sheet
    for sheet_name in sheet_names:
        # Read the sheet into a pandas DataFrame
        data_dict[sheet_name] = pd.read_excel(file, sheet_name=sheet_name)
    return data_dict

def date_to_creneau(date0: datetime.datetime, hour: pd.DataFrame, date: pd.DataFrame):
    '''
    Takes hour and date's 1-column dataframes
    and returns a dataframe of 1-column of 'créneaux'
    starting from date0
    '''
    # EXCEL IS FUCKING STUPID AND CHANGES TYPE OF DATES
    if type(date[0])==str:
        datetime_df=pd.to_datetime(date.apply(lambda x: str(x)+' ')+hour.apply(lambda x:str(x)), dayfirst=True)
    if type(date[0])==pd._libs.tslibs.timestamps.Timestamp:
        datetime_df=pd.to_datetime(date.apply(lambda x: str(x)+' ')+hour.apply(lambda x:str(x)))
    return (datetime_df-date0).dt.total_seconds().div(60*15).astype(int)+1

def creneau_to_date(date0: datetime.datetime, creneau : int):
    return date0+ datetime.timedelta(minutes=(creneau-1)*15)

# Instance sur laquelle on travaillera par la suite
INSTANCE= excel_to_dict(instance_simple)

### Dictionnaires utiles 

Trains de l'arrivée et trains du départ

In [6]:
trains_arrivee=[]
print(type(INSTANCE['Sillons arrivee']["JARR"])) #str
print(type(INSTANCE['Sillons depart']['JDEP'])) #timestamp
for index, row in INSTANCE["Sillons arrivee"].iterrows():
    id_train = row['n°TRAIN'],row['JARR']
    trains_arrivee.append(id_train)

trains_depart=[]

for index, row in INSTANCE["Sillons depart"].iterrows():
    id_train = row['n°TRAIN'],row['JDEP']
    trains_depart.append(id_train)

<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>


création des créneaux

In [9]:
def numero_jour_semaine(date_str):
    date = pd.to_datetime(date_str, format='%d/%m/%Y')
    numero_jour = date.dayofweek
    return numero_jour

def HARR_to_creneau_in_h(heure,indice):
    heure_str=''
    if indice==mini_instance:
        heure_str = heure.strftime("%H:%M")
    else:
        heure_str=heure
    heure1,minute= int(heure_str.split(':')[0]), int(heure_str.split(':')[1])    
    return heure1*4 + minute//15

In [10]:
print(HARR_to_creneau_in_h('17:46',instance_simple))

71


In [11]:
jour0=str(INSTANCE["Sillons arrivee"]["JARR"][0])
print('jour0: ',jour0)
indice_jour0=numero_jour_semaine(jour0)
# print(indice_jour0)

def creneaux_arrivee(jour0,INSTANCE,indice):
    creneaux_arrivee={}
    for index, row in INSTANCE["Sillons arrivee"].iterrows():
        delta=(datetime.strptime(row['JARR'],'%d/%m/%Y')-datetime.strptime(jour0,'%d/%m/%Y')).days
        creneaux_arrivee[(row['n°TRAIN'],row['JARR'])]=(delta//7)*96+numero_jour_semaine(row['JARR'])*96 + HARR_to_creneau_in_h(row['HARR'],indice)
    
    return creneaux_arrivee
    
def creneaux_depart(jour0,INSTANCE,indice):
    creneaux_depart={}
    for index, row in INSTANCE["Sillons depart"].iterrows():
        # ti=datetime.fromtimestamp(row['JDEP']))
        # print(row['JDEP'].to_pydatetime())
        time_str=row['JDEP']
        date=datetime.strptime('2002/09/23','%Y/%m/%d')
        if indice==mini_instance:
            time_str=row['JDEP'].strftime('%Y-%m-%d')
            date=datetime.strptime(time_str,'%Y-%m-%d')
        elif indice==instance_simple:
            time_str=row['JDEP']
            date=datetime.strptime(time_str,'%d/%m/%Y')
        else: 
            date=datetime.strptime(time_str,'%d/%m/%Y')
        delta=(date-datetime.strptime(jour0,'%d/%m/%Y')).days
        creneaux_depart[(row['n°TRAIN'],row['JDEP'])]=(delta//7)*96*7+numero_jour_semaine(row['JDEP'])*96 + HARR_to_creneau_in_h(row['HDEP'],indice)
        
    return creneaux_depart

print('creneaux_depart: ', creneaux_depart(jour0,INSTANCE,indice))
print('crenaux_arrivee: ',creneaux_arrivee(jour0,INSTANCE,indice))



jour0:  08/08/2022
creneaux_depart:  {(400006, '09/08/2022'): 159, (400016, '09/08/2022'): 187, (400110, '09/08/2022'): 179, (431035, '09/08/2022'): 156, (431091, '09/08/2022'): 121, (44203, '09/08/2022'): 167, (44241, '09/08/2022'): 149, (44249, '09/08/2022'): 108, (44806, '09/08/2022'): 128, (47245, '09/08/2022'): 153, (47289, '09/08/2022'): 140, (47821, '09/08/2022'): 185, (489447, '09/08/2022'): 120, (52672, '09/08/2022'): 160, (54003, '09/08/2022'): 111, (54051, '09/08/2022'): 118, (54086, '09/08/2022'): 111, (54231, '09/08/2022'): 152, (450226, '10/08/2022'): 221, (400014, '10/08/2022'): 271, (412124, '10/08/2022'): 204, (41258, '10/08/2022'): 202, (431035, '10/08/2022'): 252, (431091, '10/08/2022'): 217, (44203, '10/08/2022'): 263, (44241, '10/08/2022'): 245, (44249, '10/08/2022'): 204, (47205, '10/08/2022'): 248, (48215, '10/08/2022'): 256, (489044, '10/08/2022'): 200, (52672, '10/08/2022'): 256, (54003, '10/08/2022'): 207, (54053, '10/08/2022'): 254, (54231, '10/08/2022'): 248

Créneaux de l'arrivée et du départ

In [7]:
# Date de référence et date finale 
date0=str(INSTANCE["Sillons arrivee"]["JARR"][0])+' 00:00:00'
date0=pd.to_datetime(date0, dayfirst=True)
datef=pd.to_datetime(INSTANCE["Sillons depart"]["JDEP"].iloc[-1], dayfirst=True)+datetime.timedelta(hours=24)
nombre_creneaux=int((datef-date0).total_seconds()/(60*15))


# Crenau du départ et d'arrivée
INSTANCE["Sillons arrivee"]["creneau"]=date_to_creneau(date0, INSTANCE["Sillons arrivee"]["HARR"], INSTANCE["Sillons arrivee"]["JARR"])
INSTANCE["Sillons depart"]["creneau"]=date_to_creneau(date0, INSTANCE["Sillons depart"]["HDEP"], INSTANCE["Sillons depart"]["JDEP"])

creneau_arrivee={} 
for index, row in INSTANCE["Sillons arrivee"].iterrows():
    creneau_arrivee[row["n°TRAIN"],row["JARR"]]=row["creneau"]

creneau_depart={}
for index, row in INSTANCE["Sillons depart"].iterrows():
    creneau_depart[row["n°TRAIN"],row["JDEP"]]=row["creneau"]

Correspondances

In [8]:
def trains_depart_relatifs_a(train_depart)->list:
    '''
    Prend l'id d'un train du départ 
    et retourne une liste des id des trains
    d'arrivée qui le constituent
    '''
    trains_arrivee_associes = []
    for index, row in INSTANCE["Correspondances"].iterrows():
        if (row['n°Train depart'],row['Jour depart'])==train_depart :
            trains_arrivee_associes.append((row['n°Train arrivee'],row['Jour arrivee'])) 
    return trains_arrivee_associes
print(trains_depart_relatifs_a(('D1','08/08/2022')))

[('A1', '08/08/2022'), ('A2', '08/08/2022')]


Indisponibilité des machines

In [9]:
def creneaux_de_periode(indispo_str): #jour, plage_horaire
    plage_jours=nombre_creneaux//96
    creneaux = []
    if indispo_str!='0':
        periodes = indispo_str.split(';')
        for periode in periodes:
            jour, plage_horaire = periode.strip('()').split(',')
            jour = int(jour)
            jours=[jour+7*k for k in range(0,plage_jours//7+1)]
            plage_horaire = plage_horaire.strip('()').split('-')
            heure_debut, minute_debut = map(int, plage_horaire[0].split(':'))
            creneau_debut=minute_debut//15+1
            heure_fin, minute_fin = map(int, plage_horaire[1].split(':'))
            creneau_fin=minute_fin//15+1


            if (heure_debut,minute_debut)>=(heure_fin,minute_fin):
                for jour1 in jours:
                    for c in range((jour1-1)*96+heure_debut*4+creneau_debut,jour1*96):
                        if c<=nombre_creneaux:
                            creneaux.append(c)
                    for c in range(jour1*96,jour1*96+heure_fin*4+creneau_fin):
                        if c<=nombre_creneaux: creneaux.append(c)
            else:
                for jour1 in jours:
                    for c in range((jour1-1)*96+heure_debut*4+creneau_debut,(jour1-1)*96+heure_fin*4+creneau_fin):
                        if c<=nombre_creneaux: creneaux.append(c) 
    return creneaux

# data1 = INSTANCE1['Machines']
data = INSTANCE['Machines']

df = pd.DataFrame(data)

df['Indisponibilites']=df['Indisponibilites'].astype(str)
# Apply the function to each row
df['Indisponibilites_TimeSlots'] = df['Indisponibilites'].apply(creneaux_de_periode,nombre_creneaux=nombre_creneaux)

DEB_INDIS = df.loc[df['Machine'] == 'DEB', 'Indisponibilites_TimeSlots'].values[0]
FOR_INDIS = df.loc[df['Machine'] == 'FOR', 'Indisponibilites_TimeSlots'].values[0]
DEG_INDIS = df.loc[df['Machine'] == 'DEG', 'Indisponibilites_TimeSlots'].values[0]

## Modèle mathématique 
On a modélisé notre problème en partant du fait que les machines ne sont disponibles que pour un seul train et dans un seul créneau. \
De ce fait, on introduit les variables *binaires* suivantes : \
\
    - $(deb_{ic})$ : variable binaire indiquant si la machine de débranchement est utilisée \
    pour le sillion d'arrivée d'identifiant $i$ au créneau $c$ \
    - $(for_{ic})$ : variable binaire indiquant si la machine de formation est utilisée \
    pour le sillion du départ d'identifiant $i$ au créneau $c$ \
    - $(deg_{ic})$ : variable binaire indiquant si la machine de dégarage est utilisée \
    pour le sillion du départ d'identifiant $i$ au créneau $c$

Les créneaux ont une durée de $15 min$ correspondant à la durée d'une tâche machine. Les valeurs possibles des créneaux dépendent de l'instance, \
soit alors $c \in \{1,2,..., c_{max}\} $ où $c=1$ correspond au premier créneau du premier jour du travail : $[0:00 , 0:15[$ et \
$c_{max}$ le dernier créneau du dernier jour du travail : $[23:45, 0:00[$.

On note ainsi : 
$$A = \{\text{Les identifiants des trains d'arrivée}\}$$
$$D = \{\text{Les identifiants des trains du départ}\}$$
$$C= \{1,2,..., c_{max}\}$$

In [10]:
# IMPLEMNTATION DES VARIABLES 
m= Model('Modèle')
deb = {(i, c) : m.addVar(vtype = GRB.BINARY, name=f'deb_{i}_{c}') for i in trains_arrivee for c in range(1,nombre_creneaux+1)}
form = {(i, c) : m.addVar(vtype = GRB.BINARY, name=f'deb_{i}_{c}') for i in trains_depart for c in range(1,nombre_creneaux+1)}
deg = {(i, c) : m.addVar(vtype = GRB.BINARY, name=f'deg_{i}_{c}') for i in trains_depart for c in range(1,nombre_creneaux+1)}

Set parameter Username
Academic license - for non-commercial use only - expires 2025-03-12


## Contraintes
### 1. Unique débranchement, formation et dégarage
Chaque train d'arrivé subit un seul et unique débranchement, chaque train du départ subit un seul et unique débranchement et dégarage. 
$$ \forall i \in A \quad \sum_{c} deb_{i,c} = 1 $$
$$ \forall i \in D \quad \sum_{c} for_{i,c} = 1 $$
$$ \forall i \in D \quad \sum_{c} deg_{i,c} = 1 $$

In [11]:
uniqueDeb_train = {i : m.addConstr(quicksum([deb[(i, c)] for c in range(1, nombre_creneaux + 1)]) ==1, name = f'UniqueDeb_train{i}') for i in trains_arrivee}
uniqueform_train = {i : m.addConstr(quicksum([form[(i, c)] for c in range(1, nombre_creneaux + 1)]) ==1, name = f'UniqueFor_train{i}') for i in trains_depart}
uniqueDeg_train = {i : m.addConstr(quicksum([deg[(i, c)] for c in range(1, nombre_creneaux + 1)]) ==1, name = f'UniqueDeg_train{i}') for i in trains_depart}

### 2. Une seule tâche machine par créneau  
Chaque machine est exploitée par au plus un seul train sur chaque créneau.
$$ \forall c \in C \quad \sum_{i \in A} deb_{i,c} \leq 1 $$
$$ \forall c \in C \quad \sum_{i \in D} for_{i,c} \leq 1 $$
$$ \forall c \in C \quad \sum_{i \in D} deg_{i,c} \leq 1 $$

In [12]:
uniqueDeb_creneau = {c : m.addConstr(quicksum([deb[(i, c)] for i in trains_arrivee]) <=1, name = f'UniqueDeb_creneau{c}') for c in range(1, nombre_creneaux + 1)}
uniqueFor_creneau = {c : m.addConstr(quicksum([form[(i, c)] for i in trains_depart]) <=1, name = f'UniqueFor_creneau{c}') for c in range(1, nombre_creneaux + 1)}
uniqueDeg_creneau = {c : m.addConstr(quicksum([deg[(i, c)] for i in trains_depart ]) <=1, name = f'UniqueDeg_creneau{c}') for c in range(1, nombre_creneaux + 1)}


### 3. Débranchement se fait après l'arrivée du train
Pour chaque train d'arrivée $i$, le débranchement ne peut pas avoir lieu dans les créneaux précedent celui d'arrivée $c^{a}_{i}$ après une heure de tâches humaines.
$$ \forall i \in A \quad \sum_{c \space \leq \space c^{a}_{i}+4} deb_{i,c} = 0$$ 

In [13]:
deb_apres_arrivee = {i : m.addConstr(quicksum([deb[(i, c)] for c in range(1, creneau_arrivee[i] + 5)]) ==0, name = f'deb_apres_arrivee{i}') for i in trains_arrivee}


### 4. Formation d'un train de départ 
Pour chaque train du départ $t$, la formation ne peut avoir lieu qu'après débranchement de tous les trains ayant les wagons qui le constituent. \
On note : $$ \forall t \in D \quad A_t = \{\text{Les identifiants des trains d'arrivée ayant au moins un wagon du train t}\}$$
On peut exprimer la contrainte comme suit : 
$$ \forall t \in D, \space\forall c \in C, \space\forall i \in A_t \quad for_{t,c} \leq \sum_{c' \space \lt \space c} deb_{i,c'}$$ 

In [22]:
print(trains_depart_relatifs_a(trains_depart[0]))

[('A1', '08/08/2022'), ('A2', '08/08/2022')]


In [14]:
for_apres_deb = {(t,c,i) : m.addConstr(form[(t,c)]<=quicksum([deb[(i, c_p)] for c_p in range(1, c)]), name = f'for_apres_deb{t}{c}{i}') for t in trains_depart for c in range(1,nombre_creneaux+1) for i in trains_depart_relatifs_a(t)}


### 5. Dégarage après formation
Pour chaque train du départ, le dégarage ne peut avoir lieu qu'après la formation du train et $150$ minutes soit $10$ créneaux de tâches humaines.
$$ \forall t \in D, \space\forall c \in C \quad deg_{t,c} \leq \sum_{c' \space \lt \space c-10} for_{t,c'}$$ 

In [15]:
deg_apres_for = {(t,c) : m.addConstr(deg[(t,c)]<=quicksum([form[(t, c_p)] for c_p in range(1, c-9)]), name = f'deg_apres_for{t},{c}') for t in trains_depart for c in range(1,nombre_creneaux+1)}


### 6. Dégarage avant départ
Pour chaque train du départ $t$, le dégarage ne peut pas avoir lieu dans les créneaux suivants \
celui de $20$ minutes (de tâches humaines) avant le créneau du départ $c^{d}_{t}$.
$$ \forall t \in D \quad \sum_{c \space \geq \space c^{d}_{t}-2} deg_{t,c} = 0$$ 

In [16]:
deg_avant_depart = {t : m.addConstr(quicksum([deg[(t, c)] for c in range(creneau_depart[t]-2, nombre_creneaux+1)])==0, name = f'depart_apres_deg{t}') for t in trains_depart}


### 7. Disponibilités des machines
Il y a des créneaux où les machines sont indisponible, soient alors  $ \space C_{deb}, C_{for}, C_{deg} \space$  ces crénaux d'indisponibilités.
$$ \forall i \in A \quad \sum_{c \space \in \space C_{deb}} deb_{i,c} = 0 $$
$$ \forall i \in D \quad \sum_{c \space \in \space C_{for}} for_{i,c} = 0 $$
$$ \forall i \in D \quad \sum_{c \space \in \space C_{deg}} deg_{i,c} = 0 $$

In [17]:
indisponibilite_deb = {i : m.addConstr(quicksum([deb[(i, c)] for c in DEB_INDIS]) ==0, name = f'indisponibilité_deb{i}') for i in trains_arrivee }
indisponibilite_for = {i : m.addConstr(quicksum([form[(i, c)] for c in FOR_INDIS]) ==0, name = f'indisponibilité_for{i}') for i in trains_depart}
indisponibilite_deb = {i : m.addConstr(quicksum([deg[(i, c)] for c in DEG_INDIS]) ==0, name = f'indisponibilité_deg{i}') for i in trains_depart}

## Résolution

In [18]:
objfunction = 1
m.setObjective(objfunction , GRB.MINIMIZE)
m.params.outputflag = 1
m.update()

In [19]:
m.params.outputflag = 0

In [24]:
m.setParam(GRB.Param.IterationLimit, 3600)
m.optimize()
if m.status == GRB.INF_OR_UNBD:
    m.setParam(GRB.Param.Presolve, 0)
    m.optimize()

if m.status == GRB.INFEASIBLE:
    print(m.display(), "\n\tN'A PAS DE SOLUTION!!!")
elif m.status == GRB.UNBOUNDED:
    print(m.display(), "\n\tEST NON BORNÉ!!!")

Créneaux -> Dates

In [68]:
outputxlsx = 'output.xlsx'
output = {'Id tâche': [], 'Type de tâche' : [], 'Jour' : [], 'Heure début' : [], 'Durée' : [], 'Sillon' : []}
for train in trains_arrivee:
    print(train)
    print('--------------------------------------------')
    for c in range(1,nombre_creneaux+1):
        if deb[(train,c)].x==1:
            output['Id tâche'].append(f'DEB_{train[0]}_'+ creneau_to_date(date0, c).strftime('%d/%m/%Y'))
            output['Type de tâche'].append('DEB')
            output['Jour'].append(creneau_to_date(date0, c).strftime('%d/%m/%Y'))
            output['Heure début'].append(creneau_to_date(date0, c).strftime('%H:%M'))
            output['Durée'].append(15)
            output['Sillon'].append(train[0])
            print('deb de {} se fait au créneau :'.format(train))
            print(creneau_to_date(date0,c),' - ',creneau_to_date(date0,c+1))
            print('-------------------------------------------------------')


for train in trains_depart:
    print(train)
    print('--------------------------------------------')
    for c in range(1,nombre_creneaux+1):
        if form[(train,c)].x==1:
            print('For de {} se fait au créneau :'.format(train))
            output['Id tâche'].append(f'FOR_{train[0]}_'+ creneau_to_date(date0, c).strftime('%d/%m/%Y'))
            output['Type de tâche'].append('FOR')
            output['Jour'].append(creneau_to_date(date0, c).strftime('%d/%m/%Y'))
            output['Heure début'].append(creneau_to_date(date0, c).strftime('%H:%M'))
            output['Durée'].append(15)
            output['Sillon'].append(train[0])
            print(creneau_to_date(date0,c),' - ',creneau_to_date(date0,c+1))
            print('-------------------------------------------------------')
        if deg[(train,c)].x==1:
            print('deg de {} se fait au créneau :'.format(train))
            output['Id tâche'].append(f'DEG_{train[0]}_' + creneau_to_date(date0, c).strftime('%d/%m/%Y'))
            output['Type de tâche'].append('DEG')
            output['Jour'].append(creneau_to_date(date0, c).strftime('%d/%m/%Y'))
            output['Heure début'].append(creneau_to_date(date0, c).strftime('%H:%M'))
            output['Durée'].append(15)
            output['Sillon'].append(train[0])
            print(creneau_to_date(date0,c),' - ',creneau_to_date(date0,c+1))
            print('-------------------------------------------------------')
    



('A1', '08/08/2022')
--------------------------------------------
deb de ('A1', '08/08/2022') se fait au créneau :
2022-08-08 14:30:00  -  2022-08-08 14:45:00
-------------------------------------------------------
('A3', '08/08/2022')
--------------------------------------------
deb de ('A3', '08/08/2022') se fait au créneau :
2022-08-08 23:15:00  -  2022-08-08 23:30:00
-------------------------------------------------------
('A2', '08/08/2022')
--------------------------------------------
deb de ('A2', '08/08/2022') se fait au créneau :
2022-08-08 14:15:00  -  2022-08-08 14:30:00
-------------------------------------------------------
('A2', '09/08/2022')
--------------------------------------------
deb de ('A2', '09/08/2022') se fait au créneau :
2022-08-09 14:15:00  -  2022-08-09 14:30:00
-------------------------------------------------------
('A3', '09/08/2022')
--------------------------------------------
deb de ('A3', '09/08/2022') se fait au créneau :
2022-08-09 23:15:00  -  2

In [69]:

df = pd.DataFrame(output)
df.head()


,Id tâche,Type de tâche,Jour,Heure début,Durée,Sillon
0,DEB_A1_08/08/2022,DEB,08/08/2022,14:30,15,A1
1,DEB_A3_08/08/2022,DEB,08/08/2022,23:15,15,A3
2,DEB_A2_08/08/2022,DEB,08/08/2022,14:15,15,A2
3,DEB_A2_09/08/2022,DEB,09/08/2022,14:15,15,A2
4,DEB_A3_09/08/2022,DEB,09/08/2022,23:15,15,A3


In [ ]:
df.to_excel(outputxlsx, index=False)

In [26]:
print('deb de {} se fait au créneau : {}'.format(2,7))

deb de 2 se fait au créneau : 7


# Questions 
- Horraires du départ non conforme avec les créneaux machines
- Créneaux et jours ? indices dépendants 
- Il ya un choix à faire lors de la correspondance entre tâche humaine et créneaux où \
entre tâche machine et créneaux.